# Trees

## Terminology

### Node
Each node encapsulates some data and keeps track of its **children**:
![Nodes](nodes.png)

### Parent and Child
- Trees ae viewed starting from the top and branching toward bottom. 
- Every node, except for the first one, is connected to a single node above, which is referred to as a **parent** node.
- The nodes directly below and connected to the parent node are know as child nodes.
![Parent and child](parent_and_child.png)

### Root
The topmost node in the tree is called the **root** of the tree. It's the only node that has no parent:
![Root](root.png)

### Leaf
A node that has no children is called a leaf:
![Leaf](leaf.png)
 

## Implementation

In [4]:
class TreeNode<T>(val value: T) {
    // private 
    val children: MutableList<TreeNode<T>> = mutableListOf()
    
    fun add(child: TreeNode<T>) = children.add(child)
    
}

Each node is responsible for a `value` and holds references to all of its children.

In [2]:
val hot = TreeNode("Hot")
val cold = TreeNode("Cold")

val beverages = TreeNode("Beverages").run { 
    add(hot)
    add(cold)
 }

This arrangement corresponds to the following structure:
![Tree beverages](beverages.png)

## Traversal algorithms

Iterating through linear collections such as arrays or lists is straightforward. Linear collections have a clear start and end:
![Traversing arrays or lists](traversing_linear_collection.png)

Iterating through trees is a bit more complicated:
![Traversing trees](traversing_trees.png)

You traversal strategy depends on the problem you're trying to solve.

There are multiple strategies for different trees and different problems. In all of these ways you can **visit** the node and use the information into them.

In [5]:
typealias Visitor<T> = (TreeNode<T>) -> Unit

# Depth-first traversal

Depth-first traversal starts at the root node and explore the tre as far as possible along each branch before reaching a leaf and then backtracking.

In [6]:
fun <T: Any> TreeNode<T>.forEachDepthFirst(visit: Visitor<T>) {
    visit(this)
    children.forEach {
        it.forEachDepthFirst(visit)
    }
}

This simple code uses recursion to process the next node.

In [8]:
fun makeBeverageTree(): TreeNode<String> {
  val tree = TreeNode("Beverages")

  val hot = TreeNode("hot")
  val cold = TreeNode("cold")

  val tea = TreeNode("tea")
  val coffee = TreeNode("coffee")
  val chocolate = TreeNode("cocoa")

  val blackTea = TreeNode("black")
  val greenTea = TreeNode("green")
  val chaiTea = TreeNode("chai")

  val soda = TreeNode("soda")
  val milk = TreeNode("milk")

  val gingerAle = TreeNode("ginger ale")
  val bitterLemon = TreeNode("bitter lemon")

  tree.add(hot)
  tree.add(cold)

  hot.add(tea)
  hot.add(coffee)
  hot.add(chocolate)

  cold.add(soda)
  cold.add(milk)

  tea.add(blackTea)
  tea.add(greenTea)
  tea.add(chaiTea)

  soda.add(gingerAle)
  soda.add(bitterLemon)

  return tree
}

This function creates the following tree:
![A big tree](beverages_full.png)

In [9]:
val tree = makeBeverageTree()
tree.forEachDepthFirst { println(it.value) }

Beverages
hot
tea
black
green
chai
coffee
cocoa
cold
soda
ginger ale
bitter lemon
milk


# Level-order traversal

Level-order traversal visits each node of the tree based on the depth of the nodes. Starting at the root, every node on a level is visited before going to a lower level.

In [10]:
interface Queue<T> {

  fun enqueue(element: T): Boolean

  fun dequeue(): T?

  val count: Int

  val isEmpty: Boolean
    get() = count == 0

  fun peek(): T?
}

class ArrayListQueue<T> : Queue<T> {

  private val list = arrayListOf<T>()

  override fun enqueue(element: T): Boolean {
    list.add(element)
    return true
  }

  override fun dequeue(): T? {
    return if (isEmpty) null else list.removeAt(0)
  }

  override val count: Int
    get() = list.size

  override fun peek(): T? = list.getOrNull(0)

  override fun toString(): String {
    return list.toString()
  }
}

In [11]:
fun <T: Any> TreeNode<T>.forEachLevelOrder(visit: Visitor<T>) {
    visit(this)
    val queue = ArrayListQueue<TreeNode<T>>()
    children.forEach { queue.enqueue(it) }
    
    var node = queue.dequeue()
    while (node != null) {
        visit(node)
        node.children.forEach { queue.enqueue(it) }
        node = queue.dequeue()
    }
}

`forEachLevelOreder` visits each of the nodes in level-order:
![Level-order traversal](level-order.png)
You start visiting the current node and putting all its children into the queue. Then you start consuming the queue until it's empty. Every time you visit a node, you also put all its children into the queue. This ensures that all nodes at the same level are visited one after the other.

In [12]:
val tree = makeBeverageTree()
tree.forEachLevelOrder { println(it.value) }

Beverages
hot
cold
tea
coffee
cocoa
soda
milk
black
green
chai
ginger ale
bitter lemon


# Search

In [13]:
fun <T : Any> TreeNode<T>.search(value: T): TreeNode<T>? {
    var result: TreeNode<T>? = null
    forEachLevelOrder {
        if (it.value == value) {
            result = it
        }
    }
    return result
}

In [14]:
val tree = makeBeverageTree()
tree.search("ginger ale")?.let {
    println("Found node: ${it.value}")
}

tree.search("WKD Blue")?.let {
    println(it.value)
} ?: println("Couldn't find WKD Blue")

Found node: ginger ale
Couldn't find WKD Blue


Since it visits all nodes, if there are multiple matches, the last match wins. This means that you will get different objects back depending on what traversal you use.

# Challenges

## Challenge 1: Tree challenge
Print the values in a tree in an order based on their level. Nodes belonging to the same level should be printed on the same line.

My Solution:

In [35]:
fun <T : Any> TreeNode<T>.printlnForEachLevel() {
    println(this.value)
    val queue = ArrayListQueue<TreeNode<T>>()
    var currentChildCounter = children.size
    children.forEach { queue.enqueue(it) }

    var node = queue.dequeue()
    var levelChildCounter = 0
    while (node != null) {
        currentChildCounter--
        print("${node.value} ")
        levelChildCounter += node.children.size
        node.children.forEach { queue.enqueue(it) }
        node = queue.dequeue()
        if (currentChildCounter == 0) {
            println()
            currentChildCounter = levelChildCounter
            levelChildCounter = 0
        }
    }
}

val tree = makeBeverageTree()
tree.printlnForEachLevel()

Beverages
hot cold 
tea coffee cocoa soda milk 
black green chai ginger ale bitter lemon 


Book Solution:

In [ ]:
fun <T: Any> TreeNode<T>.printEachLevel() {
    // 1
    val queue = ArrayListQueue<TreeNode<T>>()
    var nodesLeftInCurrentLevel = 0
    queue.enqueue(this)
    
    // 2
    while (queue.isEmpty.not()) {
        // 3
        nodesLeftInCurrentLevel = queue.count
        
        // 4
        while (nodesLeftInCurrentLevel > 0) {
            queue.dequeue()?.let { node -> 
                print("${node.value} ")
                node.children.forEach { queue.enqueue(it) }
                nodesLeftInCurrentLevel--
             } ?: break
        }
        
        // 5
        println()
    }
}

1. You use a `Queue` data structure to facilitate the level-order traversal. You also create `nodesLEftInCurrentLevel` to keep track of the number of nodes you' will need to work on before you print a new line. 
2. Your level-order traversal continues until your queue is empty.
3. Inside first `while` loop, you begin by setting `nodesLeftInCurrentLevel` to the current elements in the queue.
4. Using another `while`loop, you dequeue the first `nodesLeftInCurrentLevel` number of elements from the queue. Every element you dequeue is printed *without* establishing a new line. You also enqueue all the children of the node. 
5. At this point, you generate the new line using `println()`. In the next iteration, `nodesLeftInCurrentLevel` is updated with the count of the queue, representing the number of children from the previous iteration.

This algorithm has a time complexity of **O(n)**, also uses **O(n)** space.

In [37]:
val tree = makeBeverageTree()
tree.printEachLevel()

Beverages 
hot cold 
tea coffee cocoa soda milk 
black green chai ginger ale bitter lemon 
